In [69]:
import pandas as pd
import psycopg2
import chardet

caminho_arquivo = "/home/juniorsilvadev/engenharia-dados/engenharia-dados-com-python-e-banco-de-dados/PostgreSQL/Arquivos JSON/V_OCORRENCIA_AMPLA.json"

with open(caminho_arquivo, "rb") as f:
    raw_data = f.read(10000)
    result = chardet.detect(raw_data)
    encoding_detectado = result['encoding']

df = pd.read_json(caminho_arquivo, encoding=encoding_detectado)

In [70]:
df.head(3)

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrência,Data_da_Ocorrencia,Hora_da_Ocorrência,Municipio,UF,Regiao,Descricao_do_Tipo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,7762,201803231711488,ICON TAXI AEREO LTDA E OUTRO,Incidente,2018-03-21,20:00:00,SÃO PAULO,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,...,0.0,0.0,0.0,AW109SP,H2T,A109,3175.0,8.0,AGUSTA,verdadeiro
1,7759,201803161337187,EJ ESCOLA DE AERONAUTICA CIVIL LTDA,Acidente,2018-03-14,15:30:00,MONTES CLAROS,MG,Sudeste,COMBUSTÍVEL,...,0.0,0.0,0.0,A152,L1P,C152,757.0,2.0,CESSNA AIRCRAFT,verdadeiro
2,7758,201802271904132,AEROTEX AVIACAO AGRICOLA LTDA.,Acidente,2018-01-26,12:20:00,INACIOLÂNDIA,GO,Centro-Oeste,EXCURSÃO DE PISTA,...,0.0,0.0,0.0,T188C,L1P,C188,1997.0,1.0,CESSNA AIRCRAFT,verdadeiro


In [71]:
colunas = ['Numero_da_Ocorrencia', 'Classificacao_da_Ocorrência', 'Data_da_Ocorrencia', 'Municipio', 'UF', 'Regiao', 'Nome_do_Fabricante', 'Modelo']
df = df[colunas]
df.rename(columns={'Classificacao_da_Ocorrência': 'Classificacao_da_Ocorrencia'}, inplace=True)
df.columns = df.columns.str.strip()

In [72]:
dbname   = 'anac_db'
user     = 'postgres'
password = 'root'
host     = 'localhost'
port     = '5432'

try:
    conexao = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)

    cursor = conexao.cursor()
    conexao.commit()
except Exception as e:
    print('Error ao conectar com banco de dados:', e)

In [73]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS anac_mapeamento (
        ID int,
        Classificacao_da_Ocorrencia VARCHAR(50),
        Dt_da_Ocorrencia DATE,
        Municipio VARCHAR(50),
        UF VARCHAR(30),
        Regiao VARCHAR(50),
        Fabricante VARCHAR(50),
        Modelo VARCHAR(30)
    );
"""

try:
    cursor.execute(create_table_query)
    conexao.commit()
except Exception as e:
    print('Error ao criar a tabela', e)
    conexao.rollback()

Quando for adicionar uma nova coluna, precisa fazer um transact adicionando uma nova coluna depois da conexão do banco de dados

In [74]:
cursor.execute("ALTER TABLE anac_mapeamento ADD Modelo VARCHAR(30)")

In [75]:
cursor.execute("TRUNCATE TABLE anac_mapeamento")

In [76]:
insert_query = """
    INSERT INTO anac_mapeamento (
        ID, Classificacao_da_Ocorrencia, Dt_da_Ocorrencia, Municipio, UF, Regiao, Fabricante, Modelo
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""

try:
    for index, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Numero_da_Ocorrencia'],
            row['Classificacao_da_Ocorrencia'],
            row['Data_da_Ocorrencia'],
            row['Municipio'],
            row['UF'],
            row['Regiao'],
            row['Nome_do_Fabricante'],
            row['Modelo']
        ))
    
    conexao.commit()
    print('Carga de dados inserida com sucesso')
except Exception as e:
    print('Error ao inserir dados:', e)
    conexao.rollback()
finally:
    cursor.close()
    conexao.close()

Carga de dados inserida com sucesso
